In [6]:
# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_format_inputs_to_ChatGPT_models.ipynb
# https://stackoverflow.com/questions/77285102/how-to-format-a-few-shot-prompt-for-gpt4-chat-completion-api

In [7]:
# pip install --upgrade openai

In [8]:
import sys, os
p = os.path.abspath('../..')
# p = p+r'\config'
sys.path.insert(1, p)


In [9]:
import warnings


import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import config.config as conf

import json
from itertools import chain
warnings.filterwarnings("ignore", category=DeprecationWarning)
# https://wikidocs.net/233348

In [10]:
# import the OpenAI Python library for calling the OpenAI API
from openai import OpenAI
import os

client = OpenAI(api_key= conf.OEPN_AI_KEY)

In [11]:
# # Example OpenAI Python library request
# MODEL = "gpt-4o-2024-08-06"
# response = client.chat.completions.create(
#     model=MODEL,
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": "Explain asynchronous programming in the style of the pirate Blackbeard."},
#     ],
#     temperature=0,
# )

In [ ]:
# print(json.dumps(json.loads(response.model_dump_json()), indent=4))

NameError: name 'response' is not defined

In [ ]:
df = pd.read_csv('../../../data/part_one_q_output_code_y.csv', index_col = 0)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/part_one_q_output_code_y.csv'

In [14]:
df['question'][0]

NameError: name 'df' is not defined

In [15]:
# sys_promt="""\nYou are an expert in analyzing and categorizing the "Difficulty Level" of Python-related programming questions based on examples below. 
# Questions are sourced from the "StackOverflow" community using the keyword "Python."\n"""

In [ ]:
# user_prompt_front = """\nFor the given (target) post that is marked by
#     <target_post> </target_post>, please let me know the
#     "Difficulty Level" of the target post, where the
#     "Difficulty Level" can be one of the followings: (0) Basic, (1)
#     Intermediate, and (2) Advanced\n"""

In [ ]:
# user_prompt_back = """\nplease output only the "Difficulty Level" of the target post, where it can only be one of, (0) Basic, (1)
#     Intermediate, and (2) Advanced, 
#     and mark '0' if it is basic, '1' if is is intermediate, and '2' if it is advanced question. 
#     Please refer to the samples below that may be helpful to measure the baseline of the difficulty labels, where each
#     of the sample is delimited by "----"\n
#     """

In [ ]:
diff_dict = {'Difficulty Level : Basic':0 ,
 'Difficulty Level : Intermediate':1, 
 'Difficulty Level : Advanced':2}

In [ ]:
df['answer_encode'] = df['answer'].apply(lambda x : diff_dict[x])

In [ ]:
df.head()

,id,question,answer,answer_encode
0,74693871,<Title>Why joblib is not recommended when savi...,Difficulty Level : Advanced,2
1,74700764,<Title>replace multiple strings in a list with...,Difficulty Level : Basic,0
2,74727945,<Title>pandas unit test AssertionError: DataFr...,Difficulty Level : Intermediate,1
3,74751254,<Title>Removing all duplicate images with diff...,Difficulty Level : Intermediate,1
4,74760874,<Title>How do I extract a binary pattern from ...,Difficulty Level : Intermediate,1


In [ ]:
diff_idx = {x : list(df[df['answer']==x].index) for x in list(diff_dict.keys())}

In [ ]:
diff_s_idx = {}
for key, value in diff_idx.items():
    print(key, value)
    dic_col = f'{key}_sample_idx'
    diff_population = value
    diff_s_idx[dic_col] = np.random.choice(diff_population, size=3, replace=False)

Difficulty Level : Basic [1, 7, 11, 16, 17, 18, 21, 22, 29, 31, 32, 33, 39, 41, 43, 44, 54, 55, 56, 57, 60, 65, 67, 69, 70, 72, 73]
Difficulty Level : Intermediate [2, 3, 4, 5, 6, 8, 9, 10, 12, 14, 19, 20, 23, 24, 25, 26, 27, 28, 30, 34, 35, 36, 38, 40, 42, 45, 46, 47, 48, 50, 51, 52, 53, 58, 59, 61, 63, 64, 68, 71]
Difficulty Level : Advanced [0, 13, 15, 37, 49, 62, 66, 74, 75]


In [ ]:
diff_s_idx

{'Difficulty Level : Basic_sample_idx': array([16,  7, 31]),
 'Difficulty Level : Intermediate_sample_idx': array([59, 50, 35]),
 'Difficulty Level : Advanced_sample_idx': array([74,  0, 15])}

In [ ]:
fewshot_q_id = list(chain.from_iterable(diff_s_idx.values()))


In [ ]:
eval_q_id =np.setdiff1d(list(df.index), fewshot_q_id)

In [ ]:
examples = []
for idx in fewshot_q_id:
# for idx in [19]:
    temp_dict = {"question" : str(df.loc[idx, 'question']),
                 "answer"   : str(df.loc[idx, 'answer_encode'])}
    examples.append(temp_dict)

In [ ]:
examples

[{'question': "<Title>How to add array to python dict?</Title>. <Question>Im creating dict from database entries:\n```result = []\n\nfor row in rows:\n    d = dict()\n    d['first'] = row[0]\n    d['second'] = row[1]\n\nresult.append(json.dumps(d, indent=3, default=str))\n\n```\nresult:\n```{'first': 1, 'second': 2 }\n```\nand everything looks nice but I want to add array to this dict and it should looks like below:\n```{'first': 1, 'second': 2, 'third': [{'somekey': row[2]}] }\n```\nand I dont know how to handle it\n```result = []\n\nfor row in rows:\n    d = dict()\n    d['first'] = row[0]\n    d['second'] = row[1]\n    d['third'] = []\n    d['third'][somekey] = row[2]\n\nresult.append(json.dumps(d, indent=3, default=str))\n \n```\nbut it doesn't work\n</Question>",
  'answer': '0'},
 {'question': '<Title>matplotlib pyplot display ticks and values which are in scientific form</Title>. <Question>I have a plot which looks like this:\n\nThe values displayed on the blue line are the cost

In [ ]:
q_prompt = """"""
a=1
for q_a in examples : 
    q_prompt = q_prompt+"""\n----\n"""
    q_prompt = q_prompt+f"Difficulty Level : {q_a['answer']}\n"
    q_prompt = q_prompt+"""<post>\n"""
    q_prompt = q_prompt+f"{q_a['question']}\n"
    q_prompt = q_prompt+"""</post>\n"""
q_prompt = q_prompt+"""----\n"""

In [ ]:
eval_q_idx= np.random.choice(eval_q_id, size=1, replace=False)

In [ ]:
target_post="""\n"""
target_post = target_post+"""<target_post>\n"""
target_post = target_post+df.loc[eval_q_idx, 'question'].values[0]+'\n'
target_post = target_post+"""</target_post>\n"""


In [ ]:
sys_promt = """
You are an expert in analyzing and categorizing the "Difficulty Level" of Python-related questions.
please let me know the
    "Difficulty Level" of the target post, where the
    "Difficulty Level" can be one of the followings: (1)
    Advanced, which is for a difficult one, (2) Intermediate,
    which is for a somewhat difficult one, and (3) Basic,
    which is an easy one.

***Instructions***

***1. Analyze the examples of questions***
- refer to the samples below that may be helpful to measure the baseline of the "Difficulty Level", where each
of the sample is delimited by "----"

***2.Measure the "Difficulty Level" of target question***
- For the given (target) post that is marked by <target_post> </target_post>
- please let me know the "Difficulty Level" of the target post, where the "Difficulty Level" can be one of the followings : (1)
    Advanced, which is for a difficult one, (2) Intermediate,
    which is for a somewhat difficult one, and (3) Basic,
    which is an easy one.

***3.Print out the "Difficulty Level"***
- no explanation is needed for "Difficulty Level"
- Expected  output
    if question == Easy or Basic or Bigginer level:
        Difficulty Level: 0
    elif question == intermediate level:
        Difficulty Level: 1
    elif question == advanced level:
        Difficulty Level: 2
        
"""

In [ ]:
# tot_prompt = sys_promt+user_prompt_front+target_post+user_prompt_back+q_prompt

tot_prompt = sys_promt+q_prompt+target_post

In [ ]:
with open("output.json", "w") as file:
    json.dump(tot_prompt, file)

In [ ]:
df.loc[eval_q_idx, :]

,id,question,answer,answer_encode
54,76774917,<Title>What python library is PIP using to dis...,Difficulty Level : Basic,0


In [ ]:
df.loc[eval_q_idx, :]

,id,question,answer,answer_encode
54,76774917,<Title>What python library is PIP using to dis...,Difficulty Level : Basic,0


In [ ]:
user_prompt = q_prompt+target_post

In [ ]:
# Example OpenAI Python library request
MODEL = "gpt-4o"
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": sys_promt},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0,
)

In [ ]:
js = json.dumps(json.loads(response.model_dump_json()), indent=4)

In [ ]:
print(js)

{
    "id": "chatcmpl-AYPesi0JnQ7tJGu93jzpICbZs9sWv",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "Difficulty Level: 0",
                "refusal": null,
                "role": "assistant",
                "audio": null,
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1732765146,
    "model": "gpt-4o-2024-08-06",
    "object": "chat.completion",
    "service_tier": null,
    "system_fingerprint": "fp_7f6be3efb0",
    "usage": {
        "completion_tokens": 5,
        "prompt_tokens": 5551,
        "total_tokens": 5556,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_tokens": 0
        },
        "prompt_tokens_details": {
            "audio_tokens": 0,
     

In [ ]:
r['choices'][0]['message']['content']

In [ ]:
r['choices']